In [1]:
#!/usr/bin/env python
#!/usr/bin/env python

import datetime
import logging


import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import datetime as dt
from datetime import timedelta, datetime
import sqlalchemy
from sqlalchemy import create_engine

import json
import requests
import pandas as pd
from six import string_types
from six.moves.urllib.parse import urlencode, urlunparse  # noqa

from datetime import date
from datetime import timedelta

In [2]:
import custome_credential
POSTGRES_ADDRESS = custome_credential.POSTGRES_ADDRESS
POSTGRES_PORT = custome_credential.POSTGRES_PORT
POSTGRES_USERNAME = custome_credential.POSTGRES_USERNAME
POSTGRES_PASSWORD = custome_credential.POSTGRES_PASSWORD
POSTGRES_DBNAME = custome_credential.POSTGRES_DBNAME

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

In [3]:
def facebook_info():
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials
    from pprint import pprint
    from datetime import datetime

    import datetime
    scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("#################", scope)
    client = gspread.authorize(creds)
    spreadsheetName = "#################"
    sheetName = '#################'  # <--- please set the sheet name here.
    spreadsheet = client.open(spreadsheetName)
    sheet = spreadsheet.worksheet(sheetName)
    data = sheet.get_all_records()
    tmp_df=pd.DataFrame(data)
    tmp_df['FB Spend']=tmp_df['FB Spend'].str.slice(start=1)
    tmp_df = tmp_df.replace(',','', regex=True)
    tmp_df['spend']=tmp_df['FB Spend']
    tmp_df['date_start'] = pd.to_datetime(pd.to_datetime(tmp_df['Date']).dt.strftime('%Y-%m-%d'))
    tmp_df=tmp_df.set_index('date_start')

    from datetime import date,timedelta, datetime
    date1 = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
    date2 = date.today().strftime('%Y-%m-%d')
    import datetime
    start = datetime.datetime.strptime(date1, '%Y-%m-%d')
    end = datetime.datetime.strptime(date2, '%Y-%m-%d')
    step = datetime.timedelta(days=1)
    date_list=[]
    while start <= end:
        date_list.append(start.date().strftime('%Y-%m-%d'))
        start += step
    df=pd.DataFrame({'date_start':date_list})
    df['date_start']=pd.to_datetime(df['date_start'])
    df=df.merge(tmp_df,how='left',on='date_start')
    return df[['date_start','spend']]



In [4]:
def tiktok_info():
    
    ACCESS_TOKEN = custome_credential.ACCESS_TOKEN
    PATH = "open_api/v1.2/reports/integrated/get/"

    Request_Params={
        "advertiser_id": custome_credential.advertiser_id,
        "service_type": "AUCTION",
        "report_type": "BASIC",
        "data_level": "AUCTION_ADVERTISER",
        "dimensions": [
             "advertiser_id",'stat_time_day'
        ],
        "metrics": [
            "spend",
            "impressions",
            'clicks',
            "reach",
            "conversion",
            'cpc',
            'cpm',
            'cost_per_1000_reached',
            "conversion_rate",
            "cost_per_conversion"

        ],
        "start_date": (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d'),
        "end_date": date.today().strftime('%Y-%m-%d'), 
        "page": 1,
        "page_size": 200
    }




    def build_url(path, query=""):
        # type: (str, str) -> str
        """
        Build request URL
        :param path: Request path
        :param query: Querystring
        :return: Request URL
        """
        scheme, netloc = "https", "ads.tiktok.com"
        return urlunparse((scheme, netloc, path, "", query, ""))

    def get(json_str):
        # type: (str) -> dict
        """
        Send GET request
        :param json_str: Args in JSON format
        :return: Response in JSON format
        """
        args = json.loads(json_str)
        query_string = urlencode({k: v if isinstance(v, string_types) else json.dumps(v) for k, v in args.items()})
        url = build_url(PATH, query_string)
        headers = {
            "Access-Token": ACCESS_TOKEN,
        }
        rsp = requests.get(url, headers=headers)
        return rsp.json()
    if __name__ == '__main__':
        print('start')
        metrics_list = Request_Params['metrics']
        metrics = json.dumps(metrics_list)
        data_level = Request_Params['data_level']
        page_size = Request_Params['page_size']
        start_date =Request_Params['start_date']
        end_date =Request_Params['end_date']
    #     start_date = (date.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
    #     end_date = (date.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
        advertiser_id = Request_Params['advertiser_id']
        service_type = Request_Params['service_type']
        report_type = Request_Params['report_type']
        page = Request_Params['page']
        dimensions_list = Request_Params['dimensions']
        dimensions = json.dumps(dimensions_list)
        # Args in JSON format
        df = pd.DataFrame([])
        my_args = "{\"metrics\": %s, \"data_level\": \"%s\", \"end_date\": \"%s\",\"page_size\": \"%s\", \"start_date\": \"%s\", \"advertiser_id\": \"%s\", \"service_type\": \"%s\", \"report_type\": \"%s\", \"page\": \"%s\", \"dimensions\": %s}" % (metrics, data_level, end_date, page_size, start_date, advertiser_id, service_type, report_type, page, dimensions)
        out_json_data=get(my_args)
        #convert to dataframe
        tiktok_df = pd.json_normalize(out_json_data['data']['list'])
        tiktok_df=tiktok_df.rename(columns={"metrics.conversion": "conversion", "metrics.cost_per_1000_reached": "cost_per_1000_reached",
                             "metrics.cpm": "cpm", "metrics.cpc": "cost_per_click","metrics.conversion_rate": "conversion_rate", 
                              "metrics.impressions": "impressions","metrics.spend": "spend", "dimensions.advertiser_id": "account_name",
                             "metrics.reach": "reach", "metrics.clicks": "clicks", "dimensions.stat_time_day": "date_start","metrics.cost_per_conversion":"cost_per_conversion"})
        tiktok_df['cost_per_1000_reached']=pd.to_numeric(tiktok_df['cost_per_1000_reached'])
        tiktok_df['date_start']=pd.to_datetime(tiktok_df['date_start'])
        tiktok_df['cpm']=pd.to_numeric(tiktok_df['cpm'])
        tiktok_df['cost_per_click']=pd.to_numeric(tiktok_df['cost_per_click'])
        tiktok_df['conversion_rate']=pd.to_numeric(tiktok_df['conversion_rate'])
        tiktok_df['cost_per_conversion']=pd.to_numeric(tiktok_df['cost_per_conversion'])
        tiktok_df['spend']=pd.to_numeric(tiktok_df['spend'])
        tiktok_df['platform']='tiktok'
        tiktok_df['account_name']=tiktok_df['account_name'].astype('string')
        
        tiktok_newdf = tiktok_df.set_index(['date_start', 'account_name'], append=True)
        tiktok_newdf=tiktok_newdf.drop(columns=['platform'])
        tiktok_df_spend=tiktok_newdf[['spend']]
        tiktok_df_spend=tiktok_df_spend.pivot_table(index='date_start', columns='account_name')
        tiktok_df_spend.columns=tiktok_df_spend.columns.to_series().str.join('-')
        tiktok_df_spend=tiktok_df_spend.rename(columns={"spend-6823073156513660934": "spend_tiktok"})
        
    return tiktok_df_spend

In [5]:
def influencer_info():
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials
    from pprint import pprint
    from datetime import datetime
    today = datetime.now()
    if today.day == 1:
        import datetime
        print("It's the first of the month!")
        today = datetime.date.today()
        first = today.replace(day=1)
        lastMonth = first - datetime.timedelta(days=1)
        print(lastMonth.strftime("%B %Y"))
        scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("credebtial_erin.json", scope)
        client = gspread.authorize(creds)
        spreadsheetName = "#################"
        sheetName = '#################'  # <--- please set the sheet name here.
        spreadsheet = client.open(spreadsheetName)
        sheet = spreadsheet.worksheet(sheetName)
        data = sheet.get_all_records()
        df=pd.DataFrame(data)

        #fillna with 0
        df=df.fillna(0)
        #get get spend for previous month
        spend=float(df[str(lastMonth.strftime("%B %Y"))].iloc[-1][1:].replace(',',''))
        tmp_df=pd.DataFrame()
        start_date=(date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
        end_date=date.today().strftime('%Y-%m-%d')
        tmp_df['date']=pd.date_range(start=start_date,end=end_date).tolist()
        tmp_df['spend_influencers_and_misc']=[None,None,None,None,None,None,spend,None]
    else:
        print('not the first day of month')
        tmp_df=pd.DataFrame()
        start_date=(date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
        end_date=date.today().strftime('%Y-%m-%d')
        tmp_df['date_start']=pd.date_range(start=start_date,end=end_date).tolist()
        tmp_df['spend_influencers_and_misc']=[None,None,None,None,None,None,None,None]

    

    return tmp_df

In [6]:
def household_acq_inday():
    order_table = '#################'
    selFields_Orders = '#################'
    customer_table = '#################'
    selFields_Customers = '#################'
    date_range_customer_table=' where first_purchase_at >= DATEADD(day,-7, GETDATE())'
    date_range_order_table=' where order_processed_at_date >= DATEADD(day,-7, GETDATE())'
    order_query = 'SELECT '+ selFields_Orders + ' FROM ' + order_table + date_range_order_table+';'
    customer_query = 'SELECT ' + selFields_Customers +  ' FROM ' + customer_table + date_range_customer_table+';'

    customer = pd.read_sql_query(customer_query, cnx)
    customer['first_purchase_at'] = customer['first_purchase_at'].dt.strftime('%Y-%m-%d')
    customer_count=customer.drop_duplicates(subset=['household_id']).groupby('first_purchase_at').count().household_id
    
    
    customer_count.index = pd.to_datetime(customer_count.index)
    customer_count=pd.DataFrame(customer_count)
    customer_count.index.names=['date_start']
    return customer_count

In [7]:
def additional_metric(df):
    df['total_spend']=(df.drop(columns=['household_id'])).sum(axis=1)
    df['cost_per_household']=df['total_spend']/df['household_id']
    return df.sort_index()

In [8]:
def binary_table_merging(t1,t2):
    merged_table=pd.merge(t1,t2,on='date_start',how='outer')
    return merged_table
    

In [9]:
def rename(df):
    return df.rename(columns={"date_start": "date",'spend-Cirkul - Jump':'spend-fb_jump','spend-Fission Energy':'spend-fb_fission','spend':'fb_spend',
                                    'spend-influencer':'spend-influencers_and_misc.','household_id':'number_of_new_households','cost_per_household':'cpa'})

In [10]:
def uploader(df,datatable):
    #credentials
    import custome_credential
    POSTGRES_ADDRESS = custome_credential.POSTGRES_ADDRESS
    POSTGRES_PORT = custome_credential.POSTGRES_PORT
    POSTGRES_USERNAME = custome_credential.POSTGRES_USERNAME
    POSTGRES_PASSWORD = custome_credential.POSTGRES_PASSWORD
    POSTGRES_DBNAME = custome_credential.POSTGRES_DBNAME
    #data base form
    postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.
                    format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD, 
                        ipaddress=POSTGRES_ADDRESS, 
                        port=POSTGRES_PORT, 
                        dbname=POSTGRES_DBNAME))
    #connect and creat engine
    cnx = create_engine(postgres_str)
    #inject dataframe to datatable
    tmp_datatable='tmp_'+datatable
    df.to_sql(tmp_datatable, con=cnx, if_exists='replace',index=False)
    
    
    
    
   
    
    #specifc name before use
    
    sql = f"""
    UPDATE {datatable} AS f
    SET 
    fb_spend=case when t.fb_spend is not null then t.fb_spend end,
    spend_tiktok=case when t.spend_tiktok is not null then t.spend_tiktok end,
    number_of_new_households=case when t.number_of_new_households is not null then t.number_of_new_households end,
    spend_influencers_and_misc=case when t.spend_influencers_and_misc is not null then t.spend_influencers_and_misc end,

    total_spend=case when t.total_spend is not null then t.total_spend end,
    cpa=case when t.cpa is not null then t.cpa end
    FROM {tmp_datatable} AS t
    WHERE f.date = t.date
    """
    pd.read_sql_query(sql, cnx)

In [11]:
if __name__ == '__main__':
    t1=facebook_info()
    t2=tiktok_info()
    t1=binary_table_merging(t1,t2)
    t2=influencer_info()
    t1=binary_table_merging(t1,t2)
    t2=household_acq_inday()
    t1=binary_table_merging(t1,t2)

    output_df=additional_metric(t1)
    output_df=pd.DataFrame(rename(output_df)).sort_values(by=['date'])
    datatable='#################'
    try:
        uploader(output_df,datatable)
    except: 
        pass
    print('pass#############')
   

start
not the first day of month
pass#############
